In [1]:
# Flatiron Health - Short Exercise

# Using SQLite with the pandasql library in Python

import pandas as pd
import pandasql as ps
from pandasql import sqldf

diagnosis = pd.read_csv("Patient_Diagnosis.csv")
treatment = pd.read_csv("Patient_Treatment.csv")

In [2]:
# Checking the formatting of the date in the Patient_Diagnosis dataset and converting it to a YYYY-MM-DD format

diagnosis['Diagnosis_Date'].head()

0     1/21/10
1      1/9/10
2     2/17/10
3     6/13/10
4    12/15/10
Name: Diagnosis_Date, dtype: object

In [3]:
from datetime import datetime
diagnosis['formatted_diagnosis_date']=pd.to_datetime(diagnosis['Diagnosis_Date'])
diagnosis['formatted_diagnosis_date'].head()

0   2010-01-21
1   2010-01-09
2   2010-02-17
3   2010-06-13
4   2010-12-15
Name: formatted_diagnosis_date, dtype: datetime64[ns]

In [4]:
# Same operation in the Patient_Treatment dataset
treatment['formatted_treatment_date']=pd.to_datetime(treatment['Treatment_Date'])

In [5]:
# Question 1 : How many patients does the clinic have for each cancer type?

# Number of patients by cancer type

q1 = ''' SELECT diagnosis AS "Cancer Type", Count(distinct(Patient_ID)) AS "# of Patients"
        FROM diagnosis 
        GROUP BY diagnosis
        ; '''
sqldf(q1, locals())

,Cancer Type,# of Patients
0,Breast Cancer,30
1,Colon Cancer,12


In [6]:
# Checking the total number of patients

q2 = ''' SELECT COUNT(DISTINCT (diagnosis.Patient_ID)) AS "# of Patients"
         FROM diagnosis
         
         '''
sqldf(q2, locals())

,# of Patients
0,37


In [7]:
# Checking the number of patients treated for multiple cancers

q3 = ''' SELECT diagnosis.Patient_ID, Count(distinct(diagnosis)) AS "# of Diagnosis"
        FROM diagnosis 
        group by patient_ID
        HAVING Count(distinct(diagnosis))>1
        
         '''
sqldf(q3, locals())


,Patient_ID,# of Diagnosis
0,3757,2
1,6877,2
2,6889,2
3,6922,2
4,7230,2


In [8]:
# 5 out of 37, so 14% of the patients are treated for 2 cancers. 

In [9]:
# Question 2: How long after being diagnosed do patients start treatment?

# Calculating the number of days between the diagnosis date and start of the treatment

q4 = '''SELECT diagnosis.Patient_ID, formatted_diagnosis_date,min(formatted_treatment_date),
julianday(min(formatted_treatment_date))-julianday(formatted_diagnosis_date) AS "# of days before treatment"
FROM diagnosis 
JOIN treatment ON treatment.Patient_ID = diagnosis.Patient_ID
GROUP BY diagnosis.Patient_ID, Diagnosis_Date
 '''

sqldf(q4, locals()).head()

,Patient_ID,formatted_diagnosis_date,min(formatted_treatment_date),# of days before treatment
0,2038,2010-01-21 00:00:00.000000,2010-01-24 00:00:00.000000,3.0
1,2120,2010-01-09 00:00:00.000000,2010-01-24 00:00:00.000000,15.0
2,2175,2010-02-17 00:00:00.000000,2010-02-21 00:00:00.000000,4.0
3,2407,2010-06-13 00:00:00.000000,2010-06-19 00:00:00.000000,6.0
4,2425,2010-12-15 00:00:00.000000,2010-12-19 00:00:00.000000,4.0


In [10]:
# Checking how many patients started a treatment before being dignosed for a cancer type 
# (since we have patients with multiple cancers)

q5 = '''SELECT diagnosis.Patient_ID, drug_code,formatted_diagnosis_date,min(formatted_treatment_date),
julianday(min(formatted_treatment_date))-julianday(formatted_diagnosis_date) AS "# of days before treatment" 
FROM diagnosis 
JOIN treatment ON treatment.Patient_ID = diagnosis.Patient_ID
GROUP BY diagnosis.Patient_ID, Diagnosis_Date
having (julianday(min(formatted_treatment_date))-julianday(formatted_diagnosis_date))<0
 '''

sqldf(q5, locals())

,Patient_ID,Drug_Code,formatted_diagnosis_date,min(formatted_treatment_date),# of days before treatment
0,6877,C,2012-12-09 00:00:00.000000,2012-12-03 00:00:00.000000,-6.0


In [11]:
q6 = '''SELECT diagnosis.Patient_ID,diagnosis,drug_code, formatted_diagnosis_date,min(formatted_treatment_date),
julianday(min(formatted_treatment_date))-julianday(formatted_diagnosis_date) AS "# of days before treatment"
FROM diagnosis 
JOIN treatment ON treatment.Patient_ID = diagnosis.Patient_ID
WHERE treatment.Patient_ID = 6877
GROUP BY diagnosis.Patient_ID, Diagnosis_Date
 '''

sqldf(q6, locals())

,Patient_ID,Diagnosis,Drug_Code,formatted_diagnosis_date,min(formatted_treatment_date),# of days before treatment
0,6877,Colon Cancer,C,2012-11-26 00:00:00.000000,2012-12-03 00:00:00.000000,7.0
1,6877,Breast Cancer,C,2012-12-09 00:00:00.000000,2012-12-03 00:00:00.000000,-6.0


In [12]:
# Patient 6877 was treated initially for colon cancer and then diagnosed with breast cancer. Since he was prescribed 
# the same medication for both cancers, we have a negative number for the # of days before treatment. 
# Therefore, I will exclude it in my following average calculation.

In [13]:
q7 = ''' SELECT AVG(total) AS "Average # of days before treatment"
FROM
(
    SELECT diagnosis.Patient_ID, formatted_diagnosis_date,min(formatted_treatment_date),
    julianday(min(formatted_treatment_date))-julianday(formatted_diagnosis_date) AS total
    FROM diagnosis 
    JOIN treatment ON treatment.Patient_ID = diagnosis.Patient_ID
    WHERE diagnosis.Patient_ID <> 6877
    GROUP BY diagnosis.Patient_ID, Diagnosis_Date
)'''

sqldf(q7, locals())

,Average # of days before treatment
0,5.25


In [14]:
# Question 3: Which treatment plan [drug(s)] do you think would be indicated to be used as first line of treatment 
# for breast cancer?

# Writing a query to see the first drug used in the treatment of breast cancer 

q8 = '''SELECT diagnosis.Patient_ID,Diagnosis_Code,Diagnosis,min(formatted_treatment_date),Drug_Code
    FROM diagnosis 
    JOIN treatment ON treatment.Patient_ID = diagnosis.Patient_ID
    WHERE Diagnosis = "Breast Cancer" 
    
    GROUP BY diagnosis.Patient_ID'''

sqldf(q8, locals()).head()

,Patient_ID,Diagnosis_Code,Diagnosis,min(formatted_treatment_date),Drug_Code
0,2038,174.9,Breast Cancer,2010-01-24 00:00:00.000000,A
1,2120,174.1,Breast Cancer,2010-01-24 00:00:00.000000,A
2,2175,174.7,Breast Cancer,2010-02-21 00:00:00.000000,A
3,2407,174.9,Breast Cancer,2010-06-19 00:00:00.000000,A
4,2425,174.9,Breast Cancer,2010-12-19 00:00:00.000000,A


In [15]:
#Counting the patients who are using drug A as first line therapy

q9=  '''
SELECT COUNT(DISTINCT(Patient_ID))
FROM (
    SELECT diagnosis.Patient_ID,Diagnosis_Code,min(formatted_treatment_date),Drug_Code
    FROM diagnosis 
    JOIN treatment ON treatment.Patient_ID = diagnosis.Patient_ID
    WHERE Diagnosis = "Breast Cancer"
    
    GROUP BY diagnosis.Patient_ID
)
WHERE Drug_Code = "A"
'''
sqldf(q9, locals())

,COUNT(DISTINCT(Patient_ID))
0,21


In [16]:
# Using query from question 1 to calculate the percentage of breast cancer patients using drug A as first line of 
# treatment

q10=  '''
SELECT 

(CAST(( SELECT COUNT(DISTINCT(Patient_ID))
    FROM (
        SELECT diagnosis.Patient_ID,Diagnosis_Code,min(formatted_treatment_date),Drug_Code
        FROM diagnosis 
        JOIN treatment ON treatment.Patient_ID = diagnosis.Patient_ID
        WHERE Diagnosis = "Breast Cancer"
    
        GROUP BY diagnosis.Patient_ID)
    
    WHERE Drug_Code = "A") AS FLOAT)
    / 
      CAST ( (SELECT Count(distinct(Patient_ID)) 
        FROM diagnosis 
        WHERE Diagnosis = "Breast Cancer") AS FLOAT)) *100
        
        AS "% of Breast Cancer Patients using Drug A as First-Line Therapy"
    
'''
sqldf(q10, locals())

,% of Breast Cancer Patients using Drug A as First-Line Therapy
0,70.0


In [17]:
# Question 4: How do these drugs compare in terms of duration of therapy?

# For each (patient_id,drug code) pair, I will calculate the difference between the earliest and  the latest date of 
# treatment and then average it based on the drug code.

q11 = '''SELECT AVG(Utilization) AS "Average Utilization of Drug A (in days)"

FROM (
    SELECT treatment.Patient_ID, Drug_Code,min(formatted_treatment_date),max(formatted_treatment_date),
    (julianday(max(formatted_treatment_date))-julianday(min(formatted_treatment_date)))AS Utilization 
    FROM treatment
    WHERE Drug_Code="A"
    GROUP BY Patient_ID, Drug_Code)
    '''

sqldf(q11, locals())

,Average Utilization of Drug A (in days)
0,61.0


In [18]:
q12 = '''SELECT AVG(Utilization) AS "Average Utilization of Drug B (in days)"

FROM (
    SELECT treatment.Patient_ID, Drug_Code,min(formatted_treatment_date),max(formatted_treatment_date),
    (julianday(max(formatted_treatment_date))-julianday(min(formatted_treatment_date)))AS Utilization 
    FROM treatment
    WHERE Drug_Code="B"
    GROUP BY Patient_ID, Drug_Code)
    '''

sqldf(q12, locals())

,Average Utilization of Drug B (in days)
0,59.526316


In [19]:
q13 = '''SELECT AVG(Utilization) AS "Average Utilization of Drug C (in days)"

FROM (
    SELECT treatment.Patient_ID, Drug_Code,min(formatted_treatment_date),max(formatted_treatment_date),
    (julianday(max(formatted_treatment_date))-julianday(min(formatted_treatment_date)))AS Utilization 
    FROM treatment
    WHERE Drug_Code="C"
    GROUP BY Patient_ID, Drug_Code)
    '''

sqldf(q13, locals())

,Average Utilization of Drug C (in days)
0,106.6


In [20]:
# Drug C has a higher duration compared to A and B (~60 days), most likely because it is an immunotherapy drug.